In [ ]:
!pip install sovai[full]

## Pairwise Interactions

### Interactions

1. Cross-Sectional - The cross-sectional information between two tickers are compared.
   1. Mean, Std, Skew, First Diff, Zero Crossing, Turning Points
   3. Spectral Centroid, Hjorst, Hurst, Time Reversal
2. Time-Series - The time-series information between two tickers are compared.
   1. DTW (Dynamic Time Warping)
   2. Pearson Correlation. 
3. Panel Interactions - The entire dataset are compared on different axis.
   1. Tucker Decomposition


In [4]:
import sovai as sov

sov.token_auth(token="visit https://sov.ai/profile for your token")

In [5]:
# Load ratios - takes around 
df_factors = sov.data("factors/accounting"); df_factors.head()

profitability  value  solvency  cash_flow  illiquidity  \
ticker date                                                                 
A      1999-11-26         57.000  8.000    17.000     77.000       14.000   
       1999-12-03         56.000  8.000    12.000     77.000       14.000   
       1999-12-10         56.000  8.000    25.000     77.000       14.000   
       1999-12-17         56.000  8.000    10.000     77.000       14.000   
       1999-12-24         55.000  7.000    25.000     77.000       14.000   

                   momentum_long_term  momentum_medium_term  \
ticker date                                                   
A      1999-11-26              61.000                57.000   
       1999-12-03              61.000                57.000   
       1999-12-10              61.000                57.000   
       1999-12-17              61.000                57.000   
       1999-12-24              61.000                57.000   

                   short_term_reversal  price_volatility  dividend_yield  \
ticker date                                                                
A      1999-11-26               47.000            29.000          29.000   
       1999-12-03               16.000            29.000          29.000   
       1999-12-10               30.000            28.000          28.000   
       1999-12-17               20.000            11.000          11.000   
       1999-12-24               12.000            13.000          13.000   

                   earnings_consistency  small_size  low_growth  \
ticker date                                                       
A      1999-11-26                65.000       3.000      51.000   
       1999-12-03                65.000       3.000      51.000   
       1999-12-10                65.000       3.000      51.000   
       1999-12-17                65.000       3.000      51.000   
       1999-12-24                65.000       3.000      51.000   

                   low_equity_issuance  bounce_dip  accrual_growth  \
ticker date                                                          
A      1999-11-26                1.000      51.000          54.000   
       1999-12-03                1.000      51.000          54.000   
       1999-12-10                1.000      51.000          54.000   
       1999-12-17                1.000      51.000          54.000   
       1999-12-24                1.000      51.000          54.000   

                   low_depreciation_growth  current_liquidity  low_rnd  \
ticker date                                                              
A      1999-11-26                   45.000             97.000   74.000   
       1999-12-03                   45.000             96.000   74.000   
       1999-12-10                   45.000             96.000   74.000   
       1999-12-17                   45.000             96.000   74.000   
       1999-12-24                   45.000             96.000   74.000   

                   momentum  
ticker date                  
A      1999-11-26    60.000  
       1999-12-03    61.000  
       1999-12-10    61.000  
       1999-12-17    60.000  
       1999-12-24    61.000

In [72]:
df_factors.query("ticker == 'AAPL'").align(df_factors.query("ticker == 'AMZN'"), join='inner', axis=0)

(Empty CustomDataFrame
 Columns: [profitability, value, solvency, cash_flow, illiquidity, momentum_long_term, momentum_medium_term, short_term_reversal, price_volatility, dividend_yield, earnings_consistency, small_size, low_growth, low_equity_issuance, bounce_dip, accrual_growth, low_depreciation_growth, current_liquidity, low_rnd, momentum]
 Index: [],
 Empty CustomDataFrame
 Columns: [profitability, value, solvency, cash_flow, illiquidity, momentum_long_term, momentum_medium_term, short_term_reversal, price_volatility, dividend_yield, earnings_consistency, small_size, low_growth, low_equity_issuance, bounce_dip, accrual_growth, low_depreciation_growth, current_liquidity, low_rnd, momentum]
 Index: [])

In [6]:
df_slice = df_factors.select_stocks("mega").date_range("2020-01-01")

### Standard Cross-Sectional Distance (Mean, Cosine)

In [8]:
dist_matrix = df_slice.distance()

In [10]:
dist_matrix

,AAPL,ABBV,ABT,ACN,ADBE,AIG,AMD,AMZN,ASML,AVGO,AZN,BA,BABA,BAC,BHP,BP,BRK.B,BUD,C,CHL,CMCSA,COST,CRM,CSCO,CVX,DHR,DIS,GE,GOOGL,HD,HSBC,IBM,INTC,JNJ,JPM,KO,LIN,LLY,MA,MCD,META,MRK,MSFT,NFLX,NKE,NVDA,NVO,NVS,ORCL,PDD,PEP,PFE,PG,PTRCY,PYPL,QCOM,SAP,SHEL,T,TM,TMO,TSLA,TSM,TTE,UNH,V,VALE,VZ,WFC,WMT,XOM
AAPL,0.000,0.112,0.027,0.150,0.086,0.374,0.391,0.227,0.145,0.186,0.200,0.717,0.394,0.322,0.253,0.400,0.371,0.389,0.419,0.399,0.260,0.289,0.376,0.120,0.254,0.240,0.347,0.345,0.308,0.273,0.289,0.196,0.237,0.177,0.315,0.208,0.180,0.170,0.293,0.283,0.333,0.154,0.230,0.207,0.269,0.269,0.218,0.190,0.173,0.611,0.259,0.159,0.234,0.430,0.226,0.140,0.242,0.397,0.341,0.417,0.208,0.431,0.317,0.321,0.285,0.258,0.255,0.274,0.369,0.217,0.229
ABBV,0.112,0.000,0.138,0.223,0.139,0.498,0.324,0.127,0.228,0.194,0.110,0.697,0.353,0.423,0.397,0.408,0.421,0.404,0.467,0.361,0.246,0.296,0.301,0.230,0.335,0.222,0.327,0.433,0.359,0.264,0.257,0.157,0.210,0.219,0.392,0.284,0.295,0.178,0.311,0.304,0.345,0.169,0.238,0.177,0.373,0.258,0.204,0.229,0.127,0.517,0.254,0.204,0.336,0.419,0.284,0.147,0.278,0.351,0.405,0.423,0.193,0.339,0.259,0.327,0.286,0.348,0.244,0.270,0.500,0.277,0.295
ABT,0.027,0.138,0.000,0.191,0.102,0.381,0.419,0.224,0.137,0.131,0.144,0.625,0.298,0.292,0.221,0.323,0.323,0.332,0.391,0.346,0.240,0.256,0.315,0.059,0.178,0.168,0.295,0.269,0.234,0.266,0.263,0.138,0.177,0.106,0.278,0.190,0.129,0.127,0.281,0.248,0.285,0.096,0.183,0.174,0.220,0.280,0.192,0.121,0.119,0.544,0.247,0.100,0.212,0.352,0.151,0.098,0.162,0.359,0.327,0.381,0.150,0.424,0.261,0.256,0.262,0.228,0.250,0.269,0.327,0.195,0.177
ACN,0.150,0.223,0.191,0.000,0.146,0.449,0.417,0.304,0.145,0.276,0.324,0.801,0.406,0.296,0.225,0.491,0.242,0.378,0.384,0.492,0.235,0.127,0.334,0.280,0.319,0.215,0.277,0.511,0.265,0.148,0.362,0.372,0.324,0.289,0.259,0.203,0.244,0.244,0.120,0.192,0.269,0.291,0.181,0.189,0.155,0.244,0.210,0.341,0.255,0.554,0.181,0.308,0.222,0.515,0.259,0.238,0.321,0.435,0.408,0.327,0.178,0.392,0.299,0.356,0.144,0.126,0.334,0.269,0.401,0.193,0.276
ADBE,0.086,0.139,0.102,0.146,0.000,0.589,0.185,0.095,0.069,0.225,0.248,0.813,0.293,0.484,0.390,0.532,0.488,0.555,0.555,0.499,0.404,0.335,0.258,0.199,0.344,0.214,0.458,0.557,0.193,0.354,0.376,0.325,0.202,0.240,0.474,0.387,0.307,0.208,0.297,0.392,0.145,0.214,0.171,0.155,0.317,0.101,0.168,0.288,0.217,0.452,0.392,0.229,0.414,0.526,0.146,0.123,0.266,0.467,0.568,0.568,0.208,0.320,0.225,0.422,0.360,0.290,0.311,0.430,0.567,0.384,0.317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VALE,0.255,0.244,0.250,0.334,0.311,0.266,0.497,0.416,0.325,0.319,0.274,0.318,0.245,0.261,0.266,0.125,0.269,0.287,0.226,0.111,0.253,0.359,0.371,0.222,0.141,0.316,0.253,0.241,0.391,0.255,0.030,0.225,0.110,0.247,0.279,0.339,0.308,0.232,0.394,0.252,0.383,0.220,0.341,0.301,0.306,0.433,0.300,0.252,0.196,0.377,0.325,0.175,0.342,0.181,0.253,0.199,0.260,0.041,0.228,0.271,0.286,0.408,0.387,0.133,0.286,0.365,0.000,0.249,0.266,0.247,0.105
VZ,0.274,0.270,0.269,0.269,0.430,0.295,0.858,0.544,0.471,0.311,0.263,0.471,0.381,0.161,0.156,0.223,0.174,0.110,0.173,0.221,0.038,0.218,0.512,0.267,0.209,0.308,0.136,0.287,0.511,0.092,0.208,0.189,0.265,0.254,0.140,0.101,0.254,0.308,0.226,0.118,0.577,0.252,0.353,0.316,0.226,0.639,0.371,0.257,0.212,0.667,0.109,0.212,0.139,0.196,0.393,0.283,0.342,0.252,0.079,0.096,0.239,0.560,0.390,0.138,0.144,0.260,0.249,0.000,0.236,0.098,0.182
WFC,0.369,0.500,0.327,0.401,0.567,0.108,0.931,0.757,0.523,0.375,0.408,0.373,0.495,0.044,0.172,0.119,0.180,0.102,0.079,0.355,0.182,0.302,0.586,0.305,0.172,0.418,0.228,0.116,0.598,0.226,0.274,0.274,0.394,0.361,0.102,0.212,0.245,0.384,0.400,0.195,0.681,0.383,0.518,0.396,0.261,0.706,0.534,0.286,0.262,0.722,0.263,0.318,0.241,0.186,0.417,0.374,0.369,0.266,0.137,0.127,0.371,0.677,0.617,0.141,0.233,0.338,

In [12]:
dist_matrix.sort_values("AAPL")[["AAPL"]].T

,AAPL,ABT,ADBE,ABBV,CSCO,QCOM,ASML,ACN,MRK,PFE,LLY,ORCL,JNJ,LIN,AVGO,NVS,IBM,AZN,NFLX,TMO,KO,WMT,NVO,PYPL,AMZN,XOM,MSFT,PG,INTC,DHR,SAP,BHP,CVX,VALE,V,PEP,CMCSA,NVDA,NKE,HD,VZ,MCD,UNH,COST,HSBC,MA,GOOGL,JPM,TSM,TTE,BAC,META,T,GE,DIS,WFC,BRK.B,AIG,CRM,BUD,AMD,BABA,SHEL,CHL,BP,TM,C,PTRCY,TSLA,PDD,BA
AAPL,0.000,0.027,0.086,0.112,0.120,0.140,0.145,0.150,0.154,0.159,0.170,0.173,0.177,0.180,0.186,0.190,0.196,0.200,0.207,0.208,0.208,0.217,0.218,0.226,0.227,0.229,0.230,0.234,0.237,0.240,0.242,0.253,0.254,0.255,0.258,0.259,0.260,0.269,0.269,0.273,0.274,0.283,0.285,0.289,0.289,0.293,0.308,0.315,0.317,0.321,0.322,0.333,0.341,0.345,0.347,0.369,0.371,0.374,0.376,0.389,0.391,0.394,0.397,0.399,0.400,0.417,0.419,0.430,0.431,0.611,0.717


#### Other Cross-Sectional Calculations

The extracted features include `mean`, `ske`, `std` (standard deviation), `diffm` (first difference mean), `zcr` (zero crossing rate), `mac` (mean absolute change), `sc` (spectral centroid), `tp` (turning points), `acl1` (autocorrelation at lag 1), `hjorthm` (Hjorth mobility), `hurst` (Hurst exponent), `hist` (histogram mode with 5 bins), and `timerev` (time reversibility statistic). These features capture various aspects of the time series, including central tendency, variability, complexity, and temporal characteristics.

In [18]:
features = ['mean', 'skew', 'std', 'diffm', 'zcr', 'mac', 'sc', 'tp', 'acl1', 'hjorthm', 'hurst', 'hist', 'timerev']

dist_matrix_all = df_slice.distance(orient="cross-sectional", on='ticker', distance='cosine', calculations=features)

In [19]:
dist_matrix_all

,AAPL,ABBV,ABT,ACN,ADBE,AIG,AMD,AMZN,ASML,AVGO,AZN,BA,BABA,BAC,BHP,BP,BRK.B,BUD,C,CHL,CMCSA,COST,CRM,CSCO,CVX,DHR,DIS,GE,GOOGL,HD,HSBC,IBM,INTC,JNJ,JPM,KO,LIN,LLY,MA,MCD,META,MRK,MSFT,NFLX,NKE,NVDA,NVO,NVS,ORCL,PDD,PEP,PFE,PG,PTRCY,PYPL,QCOM,SAP,SHEL,T,TM,TMO,TSLA,TSM,TTE,UNH,V,VALE,VZ,WFC,WMT,XOM
AAPL,0.000,0.139,0.078,0.166,0.098,0.339,0.369,0.229,0.109,0.176,0.196,0.734,0.538,0.368,0.221,0.358,0.375,0.403,0.485,0.471,0.293,0.232,0.390,0.180,0.290,0.231,0.465,0.337,0.220,0.244,0.331,0.205,0.358,0.230,0.321,0.275,0.163,0.144,0.258,0.272,0.261,0.147,0.172,0.184,0.263,0.233,0.169,0.216,0.236,0.485,0.230,0.272,0.215,0.450,0.281,0.186,0.219,0.345,0.352,0.341,0.166,0.389,0.241,0.257,0.212,0.233,0.272,0.340,0.319,0.196,0.191
ABBV,0.139,0.000,0.252,0.221,0.159,0.440,0.334,0.208,0.192,0.284,0.131,0.799,0.504,0.505,0.371,0.415,0.443,0.452,0.583,0.516,0.323,0.278,0.369,0.363,0.440,0.282,0.456,0.428,0.309,0.268,0.384,0.287,0.384,0.351,0.443,0.427,0.334,0.213,0.315,0.368,0.277,0.240,0.236,0.168,0.341,0.234,0.195,0.309,0.197,0.440,0.246,0.374,0.355,0.505,0.293,0.223,0.317,0.383,0.476,0.418,0.181,0.364,0.227,0.354,0.256,0.357,0.315,0.382,0.523,0.286,0.289
ABT,0.078,0.252,0.000,0.272,0.195,0.395,0.519,0.267,0.180,0.167,0.218,0.626,0.391,0.353,0.235,0.317,0.328,0.324,0.428,0.419,0.239,0.285,0.467,0.091,0.166,0.196,0.375,0.337,0.237,0.308,0.356,0.137,0.229,0.139,0.331,0.213,0.130,0.167,0.276,0.275,0.322,0.136,0.199,0.251,0.279,0.366,0.236,0.131,0.257,0.560,0.263,0.165,0.199,0.339,0.252,0.191,0.176,0.400,0.310,0.340,0.208,0.497,0.292,0.246,0.266,0.228,0.373,0.269,0.325,0.185,0.236
ACN,0.166,0.221,0.272,0.000,0.197,0.387,0.451,0.382,0.174,0.344,0.292,0.826,0.570,0.326,0.225,0.458,0.293,0.396,0.415,0.549,0.300,0.163,0.424,0.368,0.388,0.293,0.380,0.511,0.263,0.146,0.407,0.419,0.470,0.384,0.351,0.288,0.300,0.298,0.143,0.209,0.286,0.333,0.210,0.229,0.162,0.281,0.245,0.385,0.337,0.518,0.157,0.419,0.252,0.548,0.330,0.317,0.356,0.451,0.428,0.318,0.215,0.415,0.292,0.379,0.140,0.176,0.380,0.358,0.349,0.205,0.283
ADBE,0.098,0.159,0.195,0.197,0.000,0.479,0.179,0.145,0.045,0.251,0.224,0.820,0.443,0.511,0.319,0.475,0.488,0.567,0.637,0.623,0.444,0.292,0.302,0.304,0.376,0.223,0.593,0.479,0.149,0.350,0.439,0.367,0.381,0.336,0.441,0.472,0.304,0.183,0.296,0.425,0.112,0.216,0.157,0.143,0.288,0.099,0.154,0.350,0.270,0.310,0.360,0.373,0.413,0.597,0.209,0.154,0.240,0.404,0.581,0.495,0.200,0.264,0.193,0.341,0.283,0.284,0.315,0.513,0.467,0.355,0.257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VALE,0.272,0.315,0.373,0.380,0.315,0.330,0.535,0.454,0.330,0.325,0.288,0.507,0.495,0.369,0.267,0.168,0.361,0.399,0.424,0.247,0.357,0.293,0.362,0.324,0.308,0.339,0.439,0.217,0.306,0.279,0.093,0.291,0.289,0.366,0.334,0.411,0.308,0.208,0.355,0.292,0.293,0.252,0.293,0.266,0.303,0.397,0.246,0.323,0.276,0.312,0.334,0.382,0.323,0.396,0.343,0.257,0.308,0.036,0.372,0.280,0.235,0.373,0.325,0.190,0.262,0.353,0.000,0.381,0.306,0.292,0.168
VZ,0.340,0.382,0.269,0.358,0.513,0.412,0.880,0.557,0.500,0.332,0.307,0.476,0.377,0.166,0.208,0.269,0.181,0.144,0.161,0.257,0.077,0.277,0.576,0.229,0.247,0.330,0.193,0.372,0.445,0.179,0.326,0.234,0.214,0.184,0.252,0.153,0.251,0.347,0.222,0.170,0.522,0.277,0.345,0.355,0.289,0.622,0.373,0.233,0.296,0.664,0.156,0.208,0.162,0.175,0.335,0.327,0.365,0.352,0.132,0.132,0.282,0.598,0.370,0.212,0.217,0.283,0.381,0.000,0.292,0.150,0.275
WFC,0.319,0.523,0.325,0.349,0.467,0.214,0.752,0.635,0.417,0.282,0.375,0.480,0.607,0.087,0.161,0.208,0.218,0.249,0.165,0.464,0.238,0.264,0.448,0.292,0.225,0.354,0.355,0.208,0.405,0.217,0.325,0.262,0.427,0.337,0.108,0.207,0.217,0.317,0.297,0.169,0.518,0.288,0.379,0.345,0.213,0.531,0.392,0.322,0.343,0.560,0.251,0.353,0.226,0.318,0.445,0.370,0.346,0.320,0.187,0.138,0.315,0.504,0.464,0.133,0.201,0.295,

In [20]:
dist_matrix_all.sort_values("AAPL")[["AAPL"]].T

,AAPL,ABT,ADBE,ASML,ABBV,LLY,MRK,LIN,TMO,ACN,NVO,MSFT,AVGO,CSCO,NFLX,QCOM,XOM,WMT,AZN,IBM,UNH,PG,NVS,SAP,GOOGL,BHP,AMZN,PEP,JNJ,DHR,COST,V,NVDA,ORCL,TSM,HD,TTE,MA,META,NKE,PFE,VALE,MCD,KO,PYPL,CVX,CMCSA,WFC,JPM,HSBC,GE,AIG,VZ,TM,SHEL,T,INTC,BP,BAC,AMD,BRK.B,TSLA,CRM,BUD,PTRCY,DIS,CHL,C,PDD,BABA,BA
AAPL,0.000,0.078,0.098,0.109,0.139,0.144,0.147,0.163,0.166,0.166,0.169,0.172,0.176,0.180,0.184,0.186,0.191,0.196,0.196,0.205,0.212,0.215,0.216,0.219,0.220,0.221,0.229,0.230,0.230,0.231,0.232,0.233,0.233,0.236,0.241,0.244,0.257,0.258,0.261,0.263,0.272,0.272,0.272,0.275,0.281,0.290,0.293,0.319,0.321,0.331,0.337,0.339,0.340,0.341,0.345,0.352,0.358,0.358,0.368,0.369,0.375,0.389,0.390,0.403,0.450,0.465,0.471,0.485,0.485,0.538,0.734


*If you don't want to use the cosine distance to calculate the matrix, you can also use `euclidean` distance:

In [22]:
dist_matrix_all = df_slice.distance(orient="cross-sectional", distance='euclidean', calculations=features)

In [23]:
dist_matrix_all.sort_values("AAPL")[["AAPL"]].T

,AAPL,ABT,ADBE,ASML,ABBV,MRK,LLY,LIN,TMO,MSFT,NVO,CSCO,AVGO,NFLX,QCOM,XOM,WMT,AZN,ACN,IBM,UNH,PG,NVS,SAP,GOOGL,BHP,PEP,JNJ,NVDA,V,ORCL,DHR,COST,TSM,HD,AMZN,MA,META,NKE,TTE,VALE,PFE,MCD,KO,PYPL,CVX,CMCSA,JPM,WFC,HSBC,VZ,AIG,TM,SHEL,T,INTC,GE,BAC,BRK.B,AMD,TSLA,CRM,BP,BUD,CHL,DIS,PTRCY,C,PDD,BABA,BA
AAPL,0.000,0.347,0.351,0.371,0.413,0.425,0.432,0.446,0.451,0.457,0.459,0.467,0.471,0.473,0.477,0.487,0.487,0.490,0.491,0.501,0.506,0.510,0.513,0.516,0.517,0.518,0.526,0.527,0.530,0.532,0.535,0.540,0.541,0.542,0.542,0.551,0.557,0.567,0.567,0.569,0.571,0.571,0.573,0.576,0.584,0.594,0.594,0.618,0.621,0.628,0.636,0.638,0.641,0.641,0.646,0.651,0.652,0.663,0.666,0.667,0.678,0.682,0.682,0.689,0.742,0.745,0.748,0.752,0.758,0.791,0.917


### Time Series Distance 

#### Pearson Correlation (fast)

You could also try `spearman` for a non-linear monotonic correlation measure.

In [33]:
dist_matrix_pearson = df_slice.distance(orient="time-series", metric="pearson")

In [34]:
dist_matrix_pearson.sort_values("AAPL")[["AAPL"]].T

,AAPL,PTRCY,CHL,XOM,TSLA,META,PYPL,AMZN,TTE,NVDA,TSM,BP,CVX,AMD,ADBE,ASML,MSFT,DHR,NFLX,BABA,GOOGL,ORCL,INTC,TMO,MRK,VZ,SAP,BA,PDD,PFE,IBM,AVGO,SHEL,T,GE,QCOM,ABT,CRM,HSBC,LIN,ACN,NVO,AIG,LLY,COST,V,AZN,BUD,MA,NVS,NKE,BHP,ABBV,WFC,MCD,PEP,DIS,TM,WMT,KO,JPM,CSCO,BRK.B,CMCSA,JNJ,PG,UNH,HD,VALE,C,BAC
AAPL,0.000,0.742,0.752,0.754,0.758,0.762,0.767,0.772,0.774,0.774,0.774,0.775,0.775,0.776,0.776,0.777,0.778,0.782,0.783,0.783,0.784,0.786,0.786,0.787,0.788,0.789,0.789,0.790,0.791,0.791,0.791,0.791,0.792,0.793,0.793,0.794,0.794,0.796,0.797,0.797,0.798,0.799,0.802,0.802,0.806,0.808,0.810,0.812,0.812,0.813,0.813,0.814,0.816,0.818,0.819,0.820,0.821,0.822,0.824,0.825,0.826,0.826,0.827,0.829,0.831,0.832,0.833,0.834,0.836,0.842,0.848


#### Dynamic Time Warping (slow)

In [29]:
dist_matrix_dtw = df_slice.distance(orient="time-series", metric="dtw")

In [30]:
dist_matrix_dtw.sort_values("AAPL")[["AAPL"]].T

,AAPL,ABT,ADBE,CSCO,QCOM,MSFT,ACN,JNJ,LIN,ASML,PG,TMO,NVS,AVGO,MRK,WMT,CHL,KO,LLY,V,ORCL,NVO,ABBV,AMZN,PEP,PFE,DHR,COST,AZN,UNH,MA,GOOGL,HD,MCD,VZ,IBM,NKE,NFLX,NVDA,BHP,CMCSA,TSM,HSBC,VALE,SAP,PYPL,META,INTC,XOM,JPM,CVX,PTRCY,BRK.B,TM,TTE,BAC,SHEL,TSLA,AMD,T,CRM,DIS,BUD,BABA,WFC,C,AIG,GE,BP,PDD,BA
AAPL,0.000,0.413,0.460,0.472,0.490,0.503,0.518,0.520,0.526,0.527,0.528,0.529,0.544,0.553,0.558,0.558,0.559,0.563,0.564,0.564,0.565,0.566,0.568,0.572,0.573,0.577,0.578,0.580,0.581,0.585,0.586,0.587,0.590,0.590,0.599,0.600,0.601,0.603,0.604,0.609,0.610,0.612,0.618,0.620,0.622,0.629,0.633,0.636,0.663,0.664,0.665,0.671,0.672,0.675,0.676,0.678,0.686,0.688,0.689,0.689,0.691,0.705,0.712,0.726,0.735,0.737,0.738,0.741,0.743,0.871,0.886


#### Various Other Measures

The distance metrics encompass a diverse range of approaches for comparing time series, capturing various aspects of similarity and difference:

1. `dtw`: Dynamic Time Warping, which allows for non-linear alignment of time series.
2. `pearson` and `spearman`: Correlation-based distances, measuring linear and rank-based relationships respectively.
3. `euclidean` and `euclidean_int`: Euclidean distances, with the latter using interpolation for different-length series.
4. `pec`: Power Envelope Correlation, capturing similarities in the energy distribution of signals.
5. `frechet`: Fréchet distance, measuring similarity between curves.
6. `kl_divergence`: Kullback-Leibler divergence, comparing probability distributions.
7. `wasserstein`: Wasserstein distance, also known as Earth Mover's distance.
8. `jaccard`: Jaccard distance, comparing set similarity.
9. `bray_curtis`: Bray-Curtis dissimilarity, often used in ecology.
10. `hausdorff`: Hausdorff distance, measuring how far two subsets of a metric space are from each other.
11. `manhattan`: Manhattan distance, summing the absolute differences.
12. `chi2`: Chi-squared distance, useful for comparing histograms.
13. `hellinger`: Hellinger distance, measuring the similarity between probability distributions.
14. `canberra`: Canberra distance, weighted version of Manhattan distance.
15. `shannon_entropy`: Distance based on Shannon entropy, measuring information content.
16. `sample_entropy` and `approx_entropy`: Complexity measures based on repeating patterns.
17. `jensen_shannon`: Jensen-Shannon divergence, measuring similarity between probability distributions.
18. `renyi_entropy` and `tsallis_entropy`: Generalized entropy measures.
19. `mutual_information`: Distance based on mutual information, measuring mutual dependence.

These metrics capture a wide range of time series characteristics, including shape, statistical properties, complexity, and information content. They allow for comprehensive comparison of time series data, accommodating different aspects of similarity and difference in temporal patterns.

In [83]:
dist_matrix_dtw = df_slice.distance(orient="time-series", metric="tsallis_entropy")

In [84]:
dist_matrix_dtw.sort_values("AAPL")[["AAPL"]].T

,AAPL,GOOGL,META,NVS,PG,MRK,KO,JNJ,HD,ORCL,PEP,WMT,AMZN,VZ,UNH,MSFT,BABA,V,TM,MA,XOM,ABBV,NVDA,IBM,CMCSA,DIS,MCD,CSCO,TSM,CVX,GE,PFE,TMO,ABT,AVGO,CRM,ADBE,BUD,ASML,LLY,BHP,T,SAP,PTRCY,NVO,LIN,COST,INTC,ACN,AZN,TTE,DHR,TSLA,BA,BP,QCOM,NFLX,NKE,PYPL,AMD,PDD,BAC,JPM,WFC,BRK.B,C,AIG,SHEL,CHL,HSBC,VALE
AAPL,0.000,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.002,0.002,0.002,0.002,0.002,0.002,0.005,0.007,0.010,0.011,0.012,0.012,0.012,0.013,0.016,0.021,0.024,0.026,0.027,0.028,0.029,0.029,0.032,0.033,0.033,0.038,0.038,0.040,0.041,0.042,0.043,0.044,0.045,0.047,0.049,0.050,0.050,0.078,0.078,0.079,0.081,0.086,0.086,0.096,0.097,0.097,0.099,0.099,0.099,0.099,0.122,0.186,0.889,0.893,0.925,0.969


### Panel Interactions

The Tucker decomposition is our only panel distance metric, it can be seen as a generalisation of the CP decomposition: it decomposes the tensor into a small core tensor and factor matrices.

In [100]:
dist_matrix_tucker = df_slice.distance(orient="panel")

Estimated rank: [48, 52, 15]


In [101]:
dist_matrix_tucker

,AAPL,ABBV,ABT,ACN,ADBE,AIG,AMD,AMZN,ASML,AVGO,AZN,BA,BABA,BAC,BHP,BP,BRK.B,BUD,C,CHL,CMCSA,COST,CRM,CSCO,CVX,DHR,DIS,GE,GOOGL,HD,HSBC,IBM,INTC,JNJ,JPM,KO,LIN,LLY,MA,MCD,META,MRK,MSFT,NFLX,NKE,NVDA,NVO,NVS,ORCL,PDD,PEP,PFE,PG,PTRCY,PYPL,QCOM,SAP,SHEL,T,TM,TMO,TSLA,TSM,TTE,UNH,V,VALE,VZ,WFC,WMT,XOM
AAPL,0.000,0.910,0.924,0.820,0.614,1.052,1.039,0.754,0.990,1.108,1.107,1.052,1.002,0.944,0.999,0.984,1.064,0.987,1.025,1.062,1.184,1.108,1.049,0.892,1.137,1.063,1.100,0.906,0.967,1.161,0.947,1.044,1.075,0.902,0.953,0.922,0.862,1.071,1.171,0.989,1.149,1.028,0.886,1.079,0.907,1.027,1.098,0.911,0.876,1.070,1.013,1.080,0.727,0.998,1.062,0.815,1.035,1.057,0.935,1.032,0.999,0.834,0.976,0.835,0.996,0.950,0.999,0.960,1.039,0.891,0.979
ABBV,0.910,0.000,1.032,0.995,0.964,0.994,0.990,1.001,1.041,0.947,1.018,0.977,1.047,0.958,1.076,0.980,1.017,1.003,0.904,0.997,0.930,0.996,0.979,0.957,1.051,0.972,0.976,1.032,1.024,0.897,0.954,1.000,0.996,0.917,1.003,1.019,1.042,0.946,0.911,1.158,1.153,1.044,1.076,1.012,1.016,0.890,0.982,0.951,1.102,0.986,0.967,0.993,0.970,0.998,1.000,1.052,0.998,0.999,1.003,1.130,0.921,1.054,0.898,1.025,0.958,1.057,0.951,0.990,1.096,0.915,0.887
ABT,0.924,1.032,0.000,0.943,1.061,0.901,0.979,0.865,0.877,0.883,1.018,1.065,0.985,0.921,1.055,1.131,0.854,1.027,1.031,1.018,0.985,1.231,1.047,0.781,1.002,0.946,0.932,0.977,1.232,0.905,1.005,0.996,1.017,0.705,1.160,1.115,0.957,1.033,0.941,0.759,1.008,0.925,1.055,0.934,0.978,0.975,1.078,0.913,1.129,0.984,1.179,0.882,0.881,0.999,0.984,1.044,1.045,0.877,0.848,1.054,0.841,0.966,1.025,1.038,1.110,0.953,1.107,1.088,0.950,1.151,1.052
ACN,0.820,0.995,0.943,0.000,0.559,0.968,0.795,1.033,1.045,1.068,1.020,1.111,1.156,1.056,0.937,1.053,0.784,1.102,0.951,0.955,0.907,0.783,0.942,0.924,0.933,0.792,0.921,1.079,1.018,0.840,1.054,0.908,0.984,1.074,1.019,0.939,0.849,1.185,0.940,0.797,0.924,1.105,1.073,1.004,0.970,1.013,1.052,1.205,0.860,0.945,1.177,0.997,1.169,1.027,1.137,0.986,1.048,1.042,0.987,1.090,1.078,1.086,1.031,1.005,0.736,0.747,0.950,1.035,0.969,1.058,0.909
ADBE,0.614,0.964,1.061,0.559,0.000,1.001,1.030,1.028,0.911,0.926,0.963,0.928,0.904,0.972,1.113,1.019,1.085,0.956,1.159,1.001,0.917,1.075,0.960,1.040,1.021,0.992,1.054,1.041,0.879,0.875,0.997,1.081,0.894,1.192,1.030,0.921,0.943,0.903,1.152,1.024,0.801,0.895,1.091,0.929,1.030,1.048,1.030,0.947,1.030,1.027,1.191,1.031,1.103,0.981,0.960,1.032,1.027,1.077,0.975,1.029,1.033,1.040,1.142,1.017,0.961,0.834,0.960,0.867,0.943,1.065,1.006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VALE,0.999,0.951,1.107,0.950,0.960,0.939,1.018,0.955,0.835,1.078,1.016,1.072,1.020,1.035,0.677,1.028,0.979,0.857,1.110,0.998,0.980,1.079,1.002,1.054,1.057,0.991,0.954,0.992,1.104,0.986,0.688,0.899,0.993,1.011,0.963,1.152,1.026,0.997,1.139,0.982,1.030,1.048,1.100,1.063,0.925,0.953,0.950,0.959,1.031,0.980,0.995,0.875,1.244,1.022,0.855,0.932,1.005,0.779,1.032,0.981,1.025,1.002,1.081,0.972,0.911,1.112,0.000,0.970,1.049,1.041,1.075
VZ,0.960,0.990,1.088,1.035,0.867,1.013,0.974,0.829,1.072,1.103,0.893,1.056,0.920,0.861,0.797,0.814,1.007,0.986,0.977,0.999,0.773,0.905,1.066,1.163,1.004,0.985,0.910,1.108,1.160,0.884,0.945,1.124,0.870,1.056,0.981,0.827,1.149,1.001,0.982,0.990,1.066,0.794,1.006,0.934,1.105,1.115,0.944,1.109,1.095,1.113,1.048,0.901,0.767,1.017,1.078,1.009,0.972,1.143,0.745,0.815,1.121,1.030,0.723,0.952,1.115,1.011,0.970,0.000,1.085,0.953,1.194
WFC,1.039,1.096,0.950,0.969,0.943,0.993,0.927,1.060,1.106,0.997,1.089,0.934,0.984,0.649,0.952,1.129,1.102,1.088,0.982,1.010,1.039,0.945,1.012,0.999,1.039,1.029,0.879,1.003,0.935,0.924,0.812,0.934,1.127,0.990,0.901,0.745,0.997,0.973,1.002,1.155,1.138,1.012,1.233,0.978,1.115,0.851,0.954,0.935,1.008,1.037,1.067,0.960,0.902,0.993,0.927,1.008,0.950,0.856,1.000,0.899,1.100,1.008,1.035,0.994,0.955,0.934,

In [102]:
dist_matrix_tucker.sort_values("AAPL")[["AAPL"]].T

,AAPL,ADBE,PG,AMZN,QCOM,ACN,TSLA,TTE,LIN,ORCL,MSFT,WMT,CSCO,JNJ,GE,NKE,ABBV,NVS,KO,ABT,T,BAC,HSBC,V,JPM,VZ,GOOGL,TSM,XOM,BP,BUD,MCD,ASML,UNH,PTRCY,VALE,TMO,BHP,BABA,PEP,C,NVDA,MRK,TM,SAP,AMD,WFC,IBM,CRM,AIG,BA,SHEL,PYPL,CHL,DHR,BRK.B,PDD,LLY,INTC,NFLX,PFE,NVO,DIS,AZN,COST,AVGO,CVX,META,HD,MA,CMCSA
AAPL,0.000,0.614,0.727,0.754,0.815,0.820,0.834,0.835,0.862,0.876,0.886,0.891,0.892,0.902,0.906,0.907,0.910,0.911,0.922,0.924,0.935,0.944,0.947,0.950,0.953,0.960,0.967,0.976,0.979,0.984,0.987,0.989,0.990,0.996,0.998,0.999,0.999,0.999,1.002,1.013,1.025,1.027,1.028,1.032,1.035,1.039,1.039,1.044,1.049,1.052,1.052,1.057,1.062,1.062,1.063,1.064,1.070,1.071,1.075,1.079,1.080,1.098,1.100,1.107,1.108,1.108,1.137,1.149,1.161,1.171,1.184


## Date Distance Matrices

Every distance calculation above was done be default accross `on="ticker"` we can also specify `on="date"`.

In [103]:
## All previous functions converted to "date"
dist_matrix_mean = df_slice.distance(on="date")
dist_matrix_cos = df_slice.distance(orient="cross-sectional", on="date", distance='cosine', calculations=features)
dist_matrix_euc = df_slice.distance(orient="cross-sectional", on="date", distance='euclidean', calculations=features)
dist_matrix_pearson = df_slice.distance(orient="time-series", on="date", metric="pearson")
dist_matrix_dtw = df_slice.distance(orient="time-series", on="date", metric="dtw")
dist_matrix_tsent = df_slice.distance(orient="time-series", on="date", metric="tsallis_entropy")
dist_matrix_tucker = df_slice.distance(orient="panel", on="date")

Estimated rank: [48, 52, 15]


In [118]:
dist_matrix_cos

,2020-01-03,2020-01-10,2020-01-17,2020-01-24,2020-01-31,2020-02-07,2020-02-14,2020-02-21,2020-02-28,2020-03-06,2020-03-13,2020-03-20,2020-03-27,2020-04-03,2020-04-10,2020-04-17,2020-04-24,2020-05-01,2020-05-08,2020-05-15,2020-05-22,2020-05-29,2020-06-05,2020-06-12,2020-06-19,2020-06-26,2020-07-03,2020-07-10,2020-07-17,2020-07-24,2020-07-31,2020-08-07,2020-08-14,2020-08-21,2020-08-28,2020-09-04,2020-09-11,2020-09-18,2020-09-25,2020-10-02,2020-10-09,2020-10-16,2020-10-23,2020-10-30,2020-11-06,2020-11-13,2020-11-20,2020-11-27,2020-12-04,2020-12-11,2020-12-18,2020-12-25,2021-01-01,2021-01-08,2021-01-15,2021-01-22,2021-01-29,2021-02-05,2021-02-12,2021-02-19,2021-02-26,2021-03-05,2021-03-12,2021-03-19,2021-03-26,2021-04-02,2021-04-09,2021-04-16,2021-04-23,2021-04-30,2021-05-07,2021-05-14,2021-05-21,2021-05-28,2021-06-04,2021-06-11,2021-06-18,2021-06-25,2021-07-02,2021-07-09,2021-07-16,2021-07-23,2021-07-30,2021-08-06,2021-08-13,2021-08-20,2021-08-27,2021-09-03,2021-09-10,2021-09-17,2021-09-24,2021-10-01,2021-10-08,2021-10-15,2021-10-22,2021-10-29,2021-11-05,2021-11-12,2021-11-19,2021-11-26,2021-12-03,2021-12-10,2021-12-17,2021-12-24,2021-12-31,2022-01-07,2022-01-14,2022-01-21,2022-01-28,2022-02-04,2022-02-11,2022-02-18,2022-02-25,2022-03-04,2022-03-11,2022-03-18,2022-03-25,2022-04-01,2022-04-08,2022-04-15,2022-04-22,2022-04-29,2022-05-06,2022-05-13,2022-05-20,2022-05-27,2022-06-03,2022-06-10,2022-06-17,2022-06-24,2022-07-01,2022-07-08,2022-07-15,2022-07-22,2022-07-29,2022-08-05,2022-08-12,2022-08-19,2022-08-26,2022-09-02,2022-09-09,2022-09-16,2022-09-23,2022-09-30,2022-10-07,2022-10-14,2022-10-21,2022-10-28,2022-11-04,2022-11-11,2022-11-18,2022-11-25,2022-12-02,2022-12-09,2022-12-16,2022-12-23,2022-12-30,2023-01-06,2023-01-13,2023-01-20,2023-01-27,2023-02-03,2023-02-10,2023-02-17,2023-02-24,2023-03-03,2023-03-10,2023-03-17,2023-03-24,2023-03-31,2023-04-07,2023-04-14,2023-04-21,2023-04-28,2023-05-05,2023-05-12,2023-05-19,2023-05-26,2023-06-02,2023-06-09,2023-06-16,2023-06-23,2023-06-30,2023-07-07,2023-07-14,2023-07-21,2023-07-28,2023-08-04,2023-08-11,2023-08-18,2023-08-25,2023-09-01,2023-09-08,2023-09-15,2023-09-22,2023-09-29,2023-10-06,2023-10-13,2023-10-20,2023-10-27,2023-11-03,2023-11-10,2023-11-17,2023-11-24,2023-12-01,2023-12-08,2023-12-15,2023-12-22,2023-12-29,2024-01-05,2024-01-12,2024-01-19,2024-01-26,2024-02-02,2024-02-09,2024-02-16,2024-02-23,2024-03-01,2024-03-08,2024-03-15,2024-03-22,2024-03-29,2024-04-05,2024-04-12,2024-04-19,2024-04-26,2024-05-03,2024-05-10,2024-05-17,2024-05-24,2024-05-31,2024-06-07,2024-06-14,2024-06-21,2024-06-28,2024-07-05
2020-01-03,0.000,0.053,0.013,0.030,0.081,0.047,0.097,0.048,0.057,0.053,0.094,0.164,0.181,0.143,0.109,0.122,0.247,0.237,0.062,0.059,0.087,0.146,0.132,0.114,0.120,0.248,0.144,0.112,0.062,0.174,0.141,0.121,0.223,0.174,0.178,0.100,0.135,0.214,0.137,0.171,0.153,0.089,0.131,0.059,0.065,0.084,0.173,0.230,0.099,0.195,0.301,0.258,0.076,0.180,0.556,0.542,0.389,0.514,0.527,0.581,0.248,0.488,0.678,0.410,0.627,0.628,0.376,0.789,0.650,0.515,0.625,0.535,0.294,0.312,0.293,0.171,0.247,0.181,0.192,0.305,0.290,0.314,0.368,0.356,0.432,0.335,0.294,0.283,0.283,0.298,0.257,0.300,0.271,0.179,0.244,0.266,0.270,0.250,0.218,0.382,0.338,0.154,0.192,0.139,0.170,0.226,0.126,0.304,0.107,0.117,0.180,0.098,0.133,0.206,0.166,0.170,0.116,0.106,0.228,0.312,0.213,0.497,0.409,0.130,0.132,0.115,0.107,0.162,0.231,0.293,0.250,0.215,0.241,0.114,0.251,0.354,0.278,0.174,0.381,0.411,0.316,0.256,0.213,0.328,0.179,0.113,0.468,0.399,0.351,0.405,0.283,0.387,0.440,0.363,0.208,0.187,0.169,0.193,0.149,0.149,0.313,0.256,0.141,0.183,0.150,0.102,0.157,0.213,0.347,0.399,0.355,0.257,0.325,0.375,0.208,0.131,0.101,0.082,0.143,0.067,0.055,0.065,0.046,0.095,0.041,0.140,0.147,0.096,0.059,0.110,0.098,0.152,0.100,0.099,0.074,0.090,0.074,0.143,0.055,0.095,0.165,0.080,0.098,0.196,0.176,0.066,0.113,0.123,0.148,0.138,0.136,0.075,0.068,0.128,0.139,0.195,0.143,0.094,0.053,0.115,0.093,0.098,0.157,0.096,0.115,0.140,0.167,0.124,0.057,0.094,0.063,0.079,

In [119]:
date = dist_matrix_cos.index.max()
dist_matrix_cos.sort_values(date)[[date]].T

,2024-07-05,2024-06-07,2024-06-21,2024-05-24,2024-05-17,2024-01-19,2023-11-10,2023-08-11,2024-06-14,2020-03-06,2020-03-13,2024-03-22,2024-05-31,2020-01-24,2020-02-21,2020-01-03,2024-03-08,2023-06-30,2024-01-26,2024-01-12,2020-01-17,2024-04-26,2024-05-10,2024-03-15,2020-02-14,2020-05-15,2024-02-23,2024-03-29,2023-11-17,2023-10-06,2023-10-20,2023-09-08,2020-07-17,2023-03-03,2020-11-06,2023-09-22,2020-04-03,2023-06-16,2020-09-04,2023-12-08,2023-09-15,2020-04-17,2020-07-10,2020-02-07,2023-10-13,2024-02-02,2024-03-01,2023-07-14,2022-01-28,2020-05-08,2024-01-05,2023-06-09,2020-02-28,2020-05-22,2024-04-12,2023-05-12,2021-12-24,2021-12-10,2020-07-31,2022-10-14,2024-06-28,2021-01-01,2023-12-15,2023-06-23,2023-12-22,2020-10-16,2020-10-30,2022-01-14,2022-02-04,2023-08-25,2024-04-19,2023-08-04,2023-09-29,2023-02-10,2021-12-31,2023-03-10,2023-12-29,2023-10-27,2022-12-23,2020-07-03,2024-02-09,2023-03-17,2023-08-18,2020-09-11,2023-11-24,2022-03-25,2020-11-13,2020-03-20,2022-12-30,2021-12-17,2023-05-26,2023-07-21,2023-12-01,2022-07-22,2023-09-01,2021-07-02,2023-01-20,2020-06-12,2022-06-03,2023-11-03,2020-09-25,2020-01-10,2023-01-06,2022-05-27,2021-10-15,2020-12-04,2023-05-05,2020-04-10,2023-07-28,2021-06-25,2022-01-07,2020-08-28,2023-01-13,2020-08-07,2023-02-24,2020-10-09,2020-03-27,2024-05-03,2022-04-01,2020-01-31,2020-08-21,2023-04-14,2020-05-29,2020-06-19,2023-05-19,2024-02-16,2024-04-05,2022-02-18,2023-07-07,2021-06-11,2021-10-22,2020-06-05,2021-10-29,2020-10-23,2022-03-18,2021-09-24,2020-10-02,2023-02-17,2020-07-24,2021-10-08,2022-12-16,2020-06-26,2022-07-15,2020-04-24,2023-06-02,2021-07-16,2022-08-19,2021-11-12,2022-05-13,2022-04-08,2022-02-25,2021-07-09,2023-04-21,2021-08-20,2021-07-23,2022-05-20,2021-11-05,2023-03-24,2021-12-03,2022-12-09,2023-04-07,2021-11-19,2021-01-08,2021-02-26,2022-11-18,2022-07-29,2023-04-28,2022-07-08,2022-10-07,2020-08-14,2021-05-21,2022-09-23,2021-08-06,2020-09-18,2021-08-27,2022-04-22,2022-03-11,2021-09-03,2022-06-24,2022-06-17,2021-04-09,2022-06-10,2021-06-04,2020-11-20,2022-08-12,2021-09-10,2022-11-04,2021-05-28,2023-03-31,2022-09-16,2020-12-11,2021-07-30,2021-06-18,2022-03-04,2022-11-25,2022-07-01,2022-02-11,2021-08-13,2021-09-17,2022-09-09,2023-02-03,2022-12-02,2022-10-28,2020-12-25,2020-12-18,2020-05-01,2021-11-26,2020-11-27,2022-01-21,2022-11-11,2023-01-27,2022-09-30,2022-10-21,2022-08-05,2021-10-01,2022-04-15,2021-04-30,2022-05-06,2021-03-05,2021-05-14,2021-03-19,2022-08-26,2022-09-02,2021-01-29,2021-02-12,2021-02-05,2021-05-07,2021-04-23,2021-03-26,2022-04-29,2021-04-02,2021-01-22,2021-02-19,2021-01-15,2021-04-16,2021-03-12
2024-07-05,0.000,0.004,0.046,0.046,0.058,0.060,0.064,0.066,0.066,0.069,0.071,0.073,0.075,0.076,0.081,0.085,0.085,0.086,0.090,0.090,0.090,0.091,0.093,0.097,0.097,0.098,0.100,0.100,0.101,0.101,0.102,0.102,0.103,0.103,0.103,0.104,0.104,0.105,0.105,0.108,0.108,0.109,0.110,0.110,0.110,0.111,0.112,0.114,0.114,0.119,0.121,0.124,0.125,0.128,0.131,0.131,0.132,0.133,0.133,0.134,0.135,0.136,0.138,0.138,0.139,0.140,0.141,0.141,0.142,0.144,0.144,0.144,0.145,0.146,0.148,0.148,0.149,0.151,0.151,0.154,0.155,0.155,0.155,0.158,0.158,0.159,0.162,0.162,0.163,0.164,0.166,0.167,0.167,0.167,0.168,0.169,0.169,0.170,0.170,0.171,0.171,0.173,0.173,0.174,0.176,0.176,0.177,0.177,0.178,0.179,0.183,0.184,0.185,0.188,0.188,0.190,0.190,0.191,0.191,0.194,0.197,0.200,0.200,0.201,0.203,0.204,0.216,0.219,0.221,0.222,0.228,0.231,0.234,0.235,0.236,0.240,0.242,0.245,0.247,0.248,0.248,0.249,0.250,0.251,0.252,0.255,0.255,0.257,0.258,0.260,0.260,0.260,0.266,0.269,0.269,0.269,0.270,0.271,0.272,0.273,0.274,0.276,0.277,0.277,0.277,0.281,0.283,0.283,0.288,0.289,0.291,0.295,0.302,0.305,0.308,0.312,0.313,0.316,0.316,0.316,0.316,0.319,0.323,0.325,0.326,0.329,0.330,0.331,0.332,0.333,0.334,0.335,0.337,0.339,0.340,0.340,0.342,0.352,0.353,0.354,0.354,0.358,0.360,0.362,0.363,0.365,0.365,0.366,0.366,0.378,0.386,0.409,0.409,0.428,0.431,0.452,0.458,0.465,0.473,0.480,0.485,0.487,0.497,0.501,0.544,0.551,0.557,0.559,0.579,0.586,0.588,0.635,